In [ ]:
! pip install -r requirements.txt

In [1]:
import os
from dotenv import load_dotenv
from jira import JIRA
from openai import OpenAI

In [2]:
load_dotenv()
JIRA_API_TOKEN = os.getenv('PERSONAL_JIRA_TOKEN')
USERNAME = os.getenv('JIRA_USERNAME')
OPENAI_KEY = os.getenv('OPENAI_KEY')

In [3]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_KEY"),
)

In [4]:
jira_url = "https://munjal97akhil.atlassian.net/"
jira = JIRA(server=jira_url,basic_auth=(USERNAME,JIRA_API_TOKEN))
if jira:
    print('Login success.')

Login success.


In [5]:
# Get the id for your board, name parameter can be used to filter results to boards that match or partially match the specified name.
# since i have just 1 board no need to filter.
jira.boards()

[<JIRA Board: name='Mavericks BLR', id=1>]

In [6]:
# To get the issues for a specific or active sprint
sprints = jira.sprints(board_id=1)
print(sprints)

[<JIRA Sprint: name='SCRUM Sprint 1', id=1>]


In [7]:
active_sprint = [sprint for sprint in sprints if sprint.state == 'active']
print(active_sprint)

[<JIRA Sprint: name='SCRUM Sprint 1', id=1>]


In [8]:
# Fetch all issues in the active sprint
jql = f'sprint = {active_sprint[0].id}'
issues = jira.search_issues(jql)

all_baklogs = {}
for issue in issues:
    comments_list = []
    comments = issue.fields.comment.comments
    if comments:
        for comment in comments:
            comments_list.append(comment.body)
    all_baklogs[issue.key] = {
        "Title" : issue.fields.summary,
        "Description" : issue.fields.description,
        "Comments" : comments_list
    }

In [9]:
all_baklogs

{'SCRUM-2': {'Title': 'summarize jira backlogs via llm',
  'Description': 'summarize all sprint backlogs using genai',
  'Comments': ['once i get all the comments figured wrt each backlog, lets connect the llm to summarize content',
   'we’ll use llama3 with ollama ']},
 'SCRUM-1': {'Title': 'Testing Jira API',
  'Description': 'Testing trial version of jira.',
  'Comments': ['authentication successfully done',
   'got the board id from project',
   'got the active sprint from board ',
   'now getting all the backlogs with descriptions and comments.']}}

In [10]:
used_model =' gpt-3.5-turbo-16k-0613'

In [11]:
system_msg = """You're an excellent summariser, who can look at jira backlogs and give summary based on title, description and comments.
Do not include any other detail like description, comments when you present the summary. Make sure to use all the comments from the given backlog.
While you summarise the comments use the phrase 'Progress so far includes'.

"""

user_msg = f"""
Jira backlogs with comments:
{all_baklogs}

"""

In [12]:
messages = [
    {
        'role': 'system',
        'content': system_msg
    }, {
        'role': 'user',
        'content': user_msg}
]

In [ ]:
# Generate a completion
response = client.chat.completions.create(
    model="gpt-3.5-turbo-16k-0613",
    messages=messages,
    max_tokens=1000,
    temperature=0.7
)

In [ ]:
print(response.get('choices')[0]['message']['content'])

In [ ]:
# Also, track the cost for summarization effort :

# Source: https://openai.com/pricing
# Prices in $ per 1000 tokens


PRICES = {
    "gpt-4-0613": {"input": 0.03, "output": 0.06},
    "gpt-3.5-turbo-0613": {"input": 0.0015, "output": 0.002},
    "gpt-4-0125-preview": {"input": 0.01, "output": 0.03},
    "gpt-4-turbo-preview": {"input": 0.01, "output": 0.03},
    "gpt-4-1106-preview": {"input": 0.01, "output": 0.03},
    "gpt-4-1106-vision-preview": {"input": 0.01, "output": 0.03},
    "gpt-4-turbo-2024-04-09": {"input": 0.01, "output": 0.03},
    "gpt-4-turbo": {"input": 0.01, "output": 0.03},
    "gpt-4": {"input": 0.03, "output": 0.06},
    "gpt-4-32k": {"input": 0.06, "output": 0.12},
    "gpt-3.5-turbo-0125": {"input": 0.0005, "output": 0.0015},
    "gpt-3.5-turbo-1106": {"input": 0.001, "output": 0.002},
    "gpt-3.5-turbo-instruct": {"input": 0.0015, "output": 0.002},
    "gpt-3.5-turbo-16k-0613": {"input": 0.003, "output": 0.004},
    "whisper-1": {"input": 0.006, "output": 0.006},
    "tts-1": {"input": 0.015, "output": 0.015},
    "tts-hd-1": {"input": 0.03, "output": 0.03},
    "text-embedding-ada-002-v2": {"input": 0.0001, "output": 0.0001},
    "text-davinci-003": {"input": 0.02, "output": 0.02},
    "text-ada-001": {"input": 0.0004, "output": 0.0004},
}


In [ ]:
input_tokens = response.get('usage')['prompt_tokens']
output_tokens = response.data.get('usage')['completion_tokens']

In [ ]:
input_cost = input_tokens * PRICES[used_model]['input'] / 1000
output_cost = output_tokens * PRICES[used_model]['output'] / 1000
cost = input_cost + output_cost
print('Cost in $ : ', cost)